TomatoLeafAI
Project Description:
TomatoLeafAI is an intelligent web application designed to detect diseases in tomato leaves using deep learning. The system leverages a Convolutional Neural Network (CNN) trained on the PlantVillage dataset to classify tomato leaf images into ten categories, including healthy leaves and nine common tomato diseases.
Users can simply upload a photo of a tomato leaf, and the model will provide:
The predicted disease class with confidence scores
Detailed information about the disease
Suggested treatment methods for managing the disease
The project integrates Gradio for an interactive web interface, making it easy for farmers, gardeners, and agricultural professionals to quickly diagnose and take action. This tool helps in reducing crop loss, improving plant health management, and promoting sustainable agriculture.
Key Features:
Real-time leaf disease detection
User-friendly interface for easy image upload
Informative guidance on disease symptoms and treatments
Powered by a deep learning CNN model for high accuracy
Technologies Used:
TensorFlow and Keras for model training and deployment
Gradio for the interactive web application
Python for backend logic and image processing

Model

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
from tensorflow.keras import layers, models
import numpy as np
from PIL import Image
import io
import ipywidgets as widgets
from IPython.display import display

ds, info = tfds.load(
    "plant_village",
    split="train",
    as_supervised=True,
    with_info=True
)

label_names = info.features['label'].names

wanted_classes = [
    "Tomato___Bacterial_spot",
    "Tomato___Early_blight",
    "Tomato___Late_blight",
    "Tomato___Leaf_Mold",
    "Tomato___Septoria_leaf_spot",
    "Tomato___Spider_mites Two-spotted_spider_mite",
    "Tomato___Target_Spot",
    "Tomato___Tomato_mosaic_virus",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    "Tomato___healthy"
]
wanted_idx = [label_names.index(c) for c in wanted_classes]
print("کلاس‌های گوجه:", wanted_classes)

def preprocess(img, lbl):
    lbl_idx = tf.where(tf.equal(lbl, wanted_idx))[0][0]
    img = tf.cast(img, tf.float32) / 255.0
    return img, tf.cast(lbl_idx, tf.int64)

filtered_ds = ds.filter(lambda img, lbl: tf.reduce_any(tf.equal(lbl, wanted_idx)))
filtered_ds = filtered_ds.map(preprocess)

total = 14799
train_size = int(0.7 * total)
val_size = int(0.15 * total)
test_size = total - train_size - val_size

train_ds = filtered_ds.take(train_size).shuffle(5000).repeat().batch(32).prefetch(tf.data.AUTOTUNE)
val_ds = filtered_ds.skip(train_size).take(val_size).repeat().batch(32).prefetch(tf.data.AUTOTUNE)
test_ds = filtered_ds.skip(train_size+val_size).take(test_size).batch(32).prefetch(tf.data.AUTOTUNE)

model = models.Sequential([
    layers.Input(shape=(None, None, 3)),
    layers.Conv2D(32, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(64, (3,3), activation='relu'),
    layers.MaxPooling2D((2,2)),

    layers.Conv2D(128, (3,3), activation='relu'),
    layers.GlobalAveragePooling2D(),

    layers.Dense(128, activation='relu'),
    layers.Dense(len(wanted_classes), activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

steps_per_epoch = train_size // 32
validation_steps = val_size // 32

history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=25,
    steps_per_epoch=steps_per_epoch,
    validation_steps=validation_steps
)

test_loss, test_acc = model.evaluate(test_ds)
print(f"Test Accuracy: {test_acc:.2f}")

uploader = widgets.FileUpload(
    accept='image/*',
    multiple=False,
    description='Upload Image'
)
display(uploader)

def predict_image(change):
    if uploader.value:
        uploaded_file = list(uploader.value.values())[0]
        content = uploaded_file['content']
        img = Image.open(io.BytesIO(content)).convert("RGB")
        img.show()

        img_array = np.array(img)/255.0
        img_array = np.expand_dims(img_array, axis=0)

        pred = model.predict(img_array)
        class_idx = np.argmax(pred, axis=1)[0]
        print(f"Predicted class: {wanted_classes[class_idx]}")

uploader.observe(predict_image, names='value')


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/1 [00:00<?, ? splits/s]

Generating train examples...: 0 examples [00:00, ? examples/s]

...

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
323/323 ━━━━━━━━━━━━━━━━━━━━ 101s 216ms/step - accuracy: 0.3538 - loss: 1.8547 - val_accuracy: 0.4570 - val_loss: 1.4525
Epoch 2/25
323/323 ━━━━━━━━━━━━━━━━━━━━ 61s 191ms/step - accuracy: 0.5986 - loss: 1.1497 - val_accuracy: 0.6730 - val_loss: 0.9089
Epoch 3/25
323/323 ━━━━━━━━━━━━━━━━━━━━ 55s 169ms/step - accuracy: 0.6973 - loss: 0.8590 - val_accuracy: 0.7491 - val_loss: 0.6992
Epoch 4/25
323/323 ━━━━━━━━━━━━━━━━━━━━ 80s 248ms/step - accuracy: 0.7343 - loss: 0.7565 - val_accuracy: 0.7523 - val_loss: 0.6830
Epoch 5/25
323/323 ━━━━━━━━━━━━━━━━━━━━ 53s 164ms/step - accuracy: 0.7733 - loss: 0.6339 - val_accuracy: 0.7699 - val_loss: 0.6540
Epoch 6/25
323/323 ━━━━━━━━━━━━━━━━━━━━ 52s 161ms/step - accuracy: 0.7850 - loss: 0.6043 - val_accuracy: 0.6920 - val_loss: 0.8369
Epoch 7/25
323/323 ━━━━━━━━━━━━━━━━━━━━ 52s 160ms/step - accuracy: 0.7999 - loss: 0.5525 - val_accuracy: 0.8401 - val_loss: 0.4639
Epoch 8/25
323/323 ━━━━━━━━━━━━━━━━━━━━ 52s 161ms/step - accuracy: 0.8389 - loss: 

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/epoch_iterator.py:160: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


FileUpload(value={}, accept='image/*', description='Upload Image')

1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step   
Predicted class: Tomato___Septoria_leaf_spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 101ms/step
Predicted class: Tomato___Septoria_leaf_spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 41ms/step
Predicted class: Tomato___Bacterial_spot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 45ms/step
Predicted class: Tomato___Spider_mites Two-spotted_spider_mite
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 43ms/step
Predicted class: Tomato___Early_blight
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
Predicted class: Tomato___healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted class: Tomato___Late_blight
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step
Predicted class: Tomato___Leaf_Mold


Connect to google drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Save into google drive

In [ ]:
model.save('/content/drive/MyDrive/plant_model.keras')
print("✅ Model saved to Google Drive (.keras format)")


Web app with Gradio

In [3]:
import gradio as gr
import tensorflow as tf
import numpy as np
from PIL import Image

model = tf.keras.models.load_model('/content/drive/MyDrive/plant_model.keras')


wanted_classes = [
    "Tomato___Bacterial_spot",
    "Tomato___Early_blight",
    "Tomato___Late_blight",
    "Tomato___Leaf_Mold",
    "Tomato___Septoria_leaf_spot",
    "Tomato___Spider_mites Two-spotted_spider_mite",
    "Tomato___Target_Spot",
    "Tomato___Tomato_mosaic_virus",
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus",
    "Tomato___healthy"
]

disease_info = {
    "Tomato___Bacterial_spot": {
        "info": "این بیماری باعث لکه‌های قهوه‌ای کوچک روی برگ‌ها می‌شود.",
        "treatment": "استفاده از سموم باکتری‌کش و حذف برگ‌های آلوده."
    },
    "Tomato___Early_blight": {
        "info": "علایم اولیه شامل لکه‌های قهوه‌ای روی برگ‌ها و ساقه است.",
        "treatment": "سم‌پاشی قارچ‌کش و تهویه مناسب گلخانه."
    },
    "Tomato___Late_blight": {
        "info": "باعث پوسیدگی برگ‌ها و میوه‌ها می‌شود.",
        "treatment": "استفاده از قارچ‌کش مناسب و جمع‌آوری قسمت‌های آلوده."
    },
    "Tomato___Leaf_Mold": {
        "info": "لکه‌های زرد روی برگ‌ها و کپک سبز روی پشت برگ.",
        "treatment": "تهویه مناسب، کاهش رطوبت و سم‌پاشی."
    },
    "Tomato___Septoria_leaf_spot": {
        "info": "لکه‌های کوچک با هاله روشن روی برگ.",
        "treatment": "حذف برگ‌های آلوده و استفاده از قارچ‌کش."
    },
    "Tomato___Spider_mites Two-spotted_spider_mite": {
        "info": "کنه‌های ریز باعث زردی و لکه‌های کوچک روی برگ‌ها می‌شوند.",
        "treatment": "استفاده از حشره‌کش مناسب و شستشو با آب."
    },
    "Tomato___Target_Spot": {
        "info": "لکه‌های گرد و سیاه روی برگ‌ها و ساقه.",
        "treatment": "سم‌پاشی قارچ‌کش و مدیریت بقایای گیاهی."
    },
    "Tomato___Tomato_mosaic_virus": {
        "info": "ویروس باعث پیچ خوردن و تغییر رنگ برگ‌ها می‌شود.",
        "treatment": "حذف گیاهان آلوده و استفاده از بذر سالم."
    },
    "Tomato___Tomato_Yellow_Leaf_Curl_Virus": {
        "info": "باعث زردی و پیچ خوردن برگ‌ها و کاهش رشد گیاه می‌شود.",
        "treatment": "کنترل شته‌ها و استفاده از گیاهان مقاوم."
    },
    "Tomato___healthy": {
        "info": "گیاه سالم است.",
        "treatment": "نگهداری شرایط محیطی مناسب."
    }
}


def predict_tomato(image, show_treatment=False):
    img = image.convert("RGB")
    img_array = np.array(img)/255.0
    img_array = np.expand_dims(img_array, axis=0)

    pred = model.predict(img_array)
    class_idx = np.argmax(pred, axis=1)[0]
    confidence = pred[0][class_idx]

    class_name = wanted_classes[class_idx]
    info_text = disease_info[class_name]["info"]
    treatment_text = disease_info[class_name]["treatment"] if show_treatment else ""

    return (
        {wanted_classes[i]: float(pred[0][i]) for i in range(len(wanted_classes))},
        f"{class_name} ({confidence*100:.1f}%)",
        info_text,
        treatment_text
    )

iface = gr.Interface(
    fn=predict_tomato,
    inputs=[gr.Image(type="pil"), gr.Checkbox(label="نمایش روش درمان")],
    outputs=[
        gr.Label(num_top_classes=10),
        gr.Textbox(label="Class"),
        gr.Textbox(label="Info"),
        gr.Textbox(label="Treatment")
    ],
    title="Tomato Disease Classifier",
    description="یک عکس از برگ گوجه آپلود کن تا مدل بیماری یا سالم بودن آن را پیش‌بینی کند. گزینه نمایش روش درمان را فعال کنید."
)

iface.launch(share=True)


Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://ae5601bb1f8b305ab3.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [1]:
from google.colab import drive
drive.mount('/content/drive')


Mounted at /content/drive


In [2]:
import os
os.listdir("/content/drive/MyDrive/")


['Getting started.pdf',
 'Colab Notebooks',
 'plant_model.keras',
 'plant_model_saved']